In [6]:
install.packages("parallel")

Installing package into 'C:/Users/Tomás García/Documents/R/win-library/3.6'
(as 'lib' is unspecified)
Warning message:
"package 'parallel' is not available (for R version 3.6.1)"Warning message:
"package 'parallel' is a base package, and should not be updated"

In [1]:
rm( list=ls() )  #Borro todos los objetos
gc()   #Garbage Collection

require("data.table")
require("rpart")
require("parallel")

ksemillas  <- c(333333, 343434, 353535, 363636, 373737) #reemplazar por las propias semillas

#------------------------------------------------------------------------------
#particionar agrega una columna llamada fold a un dataset que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30 

particionar  <- function( data,  division, agrupa="",  campo="fold", start=1, seed=NA )
{
  if( !is.na(seed) )   set.seed( seed )

  bloque  <- unlist( mapply(  function(x,y) { rep( y, x )} ,   division,  seq( from=start, length.out=length(division) )  ) )  

  data[ , (campo) :=  sample( rep( bloque, ceiling(.N/length(bloque))) )[1:.N],
          by= agrupa ]
}
#------------------------------------------------------------------------------

ArbolEstimarGanancia  <- function( semilla, param_basicos )
{
  #particiono estratificadamente el dataset
  particionar( dataset, division=c(70,30), agrupa="clase_ternaria", seed= semilla )  #Cambiar por la primer semilla de cada uno !

  #genero el modelo
  modelo  <- rpart("clase_ternaria ~ .",     #quiero predecir clase_ternaria a partir del resto
                   data= dataset[ fold==1],  #fold==1  es training,  el 70% de los datos
                   xval= 0,
                   control= param_basicos )  #aqui van los parametros del arbol

  #aplico el modelo a los datos de testing
  prediccion  <- predict( modelo,   #el modelo que genere recien
                          dataset[ fold==2],  #fold==2  es testing, el 30% de los datos
                          type= "prob") #type= "prob"  es que devuelva la probabilidad

  #prediccion es una matriz con TRES columnas, llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
  #cada columna es el vector de probabilidades 


  #calculo la ganancia en testing  qu es fold==2
  ganancia_test  <- dataset[ fold==2, 
                             sum( ifelse( prediccion[, "BAJA+2"]  >  1/60,
                                         ifelse( clase_ternaria=="BAJA+2", 59000, -1000 ),
                                         0 ) )]

  #escalo la ganancia como si fuera todo el dataset
  ganancia_test_normalizada  <-  ganancia_test / 0.3

  return( ganancia_test_normalizada )
}
#------------------------------------------------------------------------------

#Aqui se debe poner la carpeta de la computadora local
setwd("C:\\Users\\Tomás García\\Documents\\MD")   #Establezco el Working Directory
#cargo los datos

dataset  <- fread("./datasets/paquete_premium_202011.csv")

param_basicos  <- list( "cp"=         -0.5,  #complejidad minima
                        "minsplit"=  900,  #minima cantidad de registros en un nodo para hacer el split
                        "minbucket"=  440,  #minima cantidad de registros en una hoja
                        "maxdepth"=  5 ) #profundidad máxima del arbol

#Un solo llamado, con la semilla 17
ArbolEstimarGanancia( 17, param_basicos )   


#la funcion mcmapply  llama a la funcion ArbolEstimarGanancia  tantas veces como valores tenga el vector  ksemillas
ganancias  <- mcmapply( ArbolEstimarGanancia, 
                        ksemillas,   #paso el vector de semillas, que debe ser el primer parametro de la funcion ArbolEstimarGanancia
                        MoreArgs= list( param_basicos),  #aqui paso el segundo parametro
                        SIMPLIFY= FALSE,
                        mc.cores= 1 )  #se puede subir a 5 si posee Linux o Mac OS

#muestro la lista de las ganancias en testing para la particion realizada con cada semilla
ganancias

#paso la lista a vector
unlist(ganancias)

#finalmente calculo la media (promedio)  de las ganancias
mean( unlist(ganancias) )

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,508675,27.2,1142394,61.1,621925,33.3
Vcells,943596,7.2,8388608,64.0,1601596,12.3


Loading required package: data.table
Warning message:
"package 'data.table' was built under R version 3.6.3"Loading required package: rpart
Warning message:
"package 'rpart' was built under R version 3.6.3"Loading required package: parallel


[1] 8516667

[[1]]
[1] 10860000

[[2]]
[1] 10886667

[[3]]
[1] 11543333

[[4]]
[1] 10250000

[[5]]
[1] 10023333

[1] 10860000 10886667 11543333 10250000 10023333

[1] 10712667